In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score

2023-03-11 22:33:33.192575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 22:33:33.743506: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-11 22:33:35.168600: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/mmey/miniconda3/envs/wsl-tf/lib/
2023-03-11 22:33:35.168785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plu

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12843069706202085110
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22655533056
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11624346054264627706
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-03-11 22:33:38.551687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 22:33:39.160120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:33:39.191040: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:33:39.191186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-03-11 22:33:42.084269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:33:42.084513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:33:42.084585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
import ipywidgets as widgets
import io
import os
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf

X = []
y = []
image_size = 128

In [5]:
train_path = 'data/Training'
labels = os.listdir(train_path)
print(labels)

['pituitary', 'glioma', 'meningioma']


## Get all the data and label them

In [6]:
for i in labels:
    folder_path = os.path.join(train_path, i)
    for j in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, j))
        img = cv2.resize(img, (image_size, image_size))
        
        X.append(img)
        y.append(i)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
X.shape, y.shape


((3064, 128, 128, 3), (3064,))

### Shuffle data

In [9]:
X, y = shuffle(X, y, random_state = 99)
X.shape, y.shape

((3064, 128, 128, 3), (3064,))

### Turn labels into number format

In [10]:
y_new = []
for i in y:
    y_new.append(labels.index(i))

y_new[:5]

y_new = np.array(y_new)

In [11]:
## StratifiedKFold needs 1-dimensional Y array.
# Do not convert to_categorical when using StratifiedKFold
#y_new = tf.keras.utils.to_categorical(y_new)
#y_new

#### To get F1Score while training

In [12]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

### Build CNN Model with K-Fold
> We have X and y_new. We'll perform K-Fold cross validation<br>
**iterative-stratification** is a project that provides scikit-learn compatible cross validators with stratification for multilabel data.


In [ ]:
!pip install iterative-stratification

In [14]:
# from sklearn.model_selection import KFold, StratifiedKFold 
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [17]:
y_new = tf.keras.utils.to_categorical(y_new)

In [18]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# define 10-fold cross validation test harness
kfold = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
precisionscores = []
recallscores = []
f1scores = []
histories = []

for train, test in kfold.split(X, y_new):

    ## StratifiedKFold needs 1-dimensional Y array.
    # Do not convert to_categorical when using StratifiedKFold
    # instead, use to_categorical after splitting with k-fold...
    #y_new = tf.keras.utils.to_categorical(y_new)
    #y_new = tf.keras.utils.to_categorical(y_new)
    #test

    model = Sequential()
    model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (image_size, image_size, 3)))
    model.add(Conv2D(64, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3,3), activation = 'relu'))
    model.add(Conv2D(64, (3,3), activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3,3), activation = 'relu'))
    model.add(Conv2D(128, (3,3), activation = 'relu'))
    model.add(Conv2D(128, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3,3), activation = 'relu'))
    model.add(Conv2D(256, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.3))

    model.add(Dense(3, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), get_f1])

     # Fit the model
    history = model.fit(X[train], y_new[train], epochs=20, batch_size=32, verbose=0)

    # evaluate the model
    scores = model.evaluate(X[test], y_new[test], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    cvscores.append(scores[1] * 100)
    precisionscores.append(scores[2] * 100)
    recallscores.append(scores[3] * 100)
    f1scores.append(scores[4] * 100)
    histories.append(history)

print("Accuracy (mean): %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("Precision (mean): %.2f%% (+/- %.2f%%)" % (np.mean(precisionscores), np.std(precisionscores)))
print("Recall (mean): %.2f%% (+/- %.2f%%)" % (np.mean(recallscores), np.std(recallscores)))
print("F1 (mean): %.2f%% (+/- %.2f%%)" % (np.mean(f1scores), np.std(f1scores)))

2023-03-11 22:34:24.577820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:34:24.577995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:34:24.578050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:34:24.578415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 22:34:24.578431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1700] Could not ident

10/10 [==============================] - 1s 34ms/step - loss: 0.3102 - accuracy: 0.8697 - precision: 0.8693 - recall: 0.8664 - get_f1: 0.8689
accuracy: 86.97%
10/10 [==============================] - 1s 34ms/step - loss: 0.2214 - accuracy: 0.9150 - precision_1: 0.9241 - recall_1: 0.9150 - get_f1: 0.9181
accuracy: 91.50%
10/10 [==============================] - 0s 11ms/step - loss: 0.3349 - accuracy: 0.8595 - precision_2: 0.8571 - recall_2: 0.8431 - get_f1: 0.8494
accuracy: 85.95%
10/10 [==============================] - 0s 11ms/step - loss: 0.3015 - accuracy: 0.8925 - precision_3: 0.8951 - recall_3: 0.8893 - get_f1: 0.8902
accuracy: 89.25%
10/10 [==============================] - 1s 11ms/step - loss: 0.2788 - accuracy: 0.8860 - precision_4: 0.8947 - recall_4: 0.8860 - get_f1: 0.8902
accuracy: 88.60%
10/10 [==============================] - 0s 12ms/step - loss: 0.1763 - accuracy: 0.9314 - precision_5: 0.9340 - recall_5: 0.9248 - get_f1: 0.9275
accuracy: 93.14%
10/10 [===================

In [19]:
tf.keras.backend.clear_session()

In [23]:
model.save("vanilla_bt_classification.h5")

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy'] 

epochs = range(len(acc))
fig = plt.figure(figsize = (14,7))
plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.legend(loc = 'upper left')
plt.show()